In [248]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
from nltk import PorterStemmer
import json
import re
from bs4 import BeautifulSoup
import requests
import inflect
import itertools
from tqdm import tqdm
p = inflect.engine()
%matplotlib inline
pd.options.display.max_rows = 999

stop_words = set(stopwords.words('english'))

In [2]:
# Test dataframe with only yummly data

# cuisines = []
# flavors = []
# ingredients = []
# name = []
# source = []
# for filename in os.listdir('data/metadata27638'):
#     if filename.endswith('.json'):
#         data = json.load(open('data/metadata27638/'+filename))
#         cuisines.append(data['attributes']['cuisine'][0])
#         flavors.append(data['flavors'])
#         ingredientStr = ''
#         for ingred in data['ingredientLines']:
#             ingredientStr = ingredientStr + ingred
#         ingredients.append(ingredientStr)
#         name.append(data['name'])
#         source.append(data['source']['sourceDisplayName'])
        
# df = pd.DataFrame({'cuisine':cuisines,'name':name,'flavors':flavors,'ingredients':ingredients,'source':source})

In [3]:
# Reading in data and checking if name and ingredient lists exist

names = []
ingreds = []

for filename in os.listdir('data/recipe_box'):
    if filename.endswith('.json'):
        data = list(json.load(open('data/recipe_box/'+filename)).values())
        for recipe in data:
            if ('title' not in recipe.keys() or 'ingredients' not in recipe.keys()):
                continue
            names.append(recipe['title'])
            ingreds.append(recipe['ingredients'])

In [130]:
# Creating data frame with dish names and ingredients

df = pd.DataFrame({'name':names,'ingredients':ingreds})

In [131]:
df.shape

(124647, 2)

In [138]:
# Remove empty ingredient lists

df = df[df.astype('str')['ingredients']!='[]']
df.shape

(122971, 2)

In [720]:
# Example of ingredient list

df['ingredients'][100435]

['2 tablespoons extra-virgin olive oil ADVERTISEMENT',
 '2 yellow onions, diced ADVERTISEMENT',
 '5 cloves garlic, minced ADVERTISEMENT',
 '1 small sugar pumpkin, peeled, seeded, cut into 1/2-inch dice ADVERTISEMENT',
 '4 cups beef broth ADVERTISEMENT',
 '1 (16 ounce) can diced tomatoes, undrained ADVERTISEMENT',
 '1/2 teaspoon salt ADVERTISEMENT',
 '1 teaspoon ground black pepper ADVERTISEMENT',
 '1 bunch lacinato kale, stems removed, chopped ADVERTISEMENT',
 '1 pound cubed cooked ham ADVERTISEMENT',
 '1 (15 ounce) can black beans, rinsed and drained ADVERTISEMENT',
 '2 tablespoons sherry vinegar ADVERTISEMENT',
 '1 tablespoon thinly sliced sorrel ADVERTISEMENT',
 'ADVERTISEMENT']

In [719]:
clean_recipe(df['ingredients'][100435])

['olive oil',
 'yellow onion',
 'clove garlic',
 'sugar pumpkin',
 'beef broth',
 'tomato',
 'lacinato kale',
 'cubed ham',
 'black bean',
 'sherry vinegar',
 'sorrel']

In [139]:
# Example of ingredient list with ADVERTISEMENT tags

df['ingredients'][100000]

['1/2 small head green cabbage, cored and thinly sliced ADVERTISEMENT',
 '1/2 jicama, sliced into matchsticks ADVERTISEMENT',
 '1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT',
 '1/2 cup mayonnaise ADVERTISEMENT',
 '1/4 cup pineapple juice ADVERTISEMENT',
 '1 teaspoon white sugar ADVERTISEMENT',
 'hot sauce to taste ADVERTISEMENT',
 'salt and freshly ground black pepper to taste ADVERTISEMENT',
 '1/4 bunch chopped fresh cilantro ADVERTISEMENT',
 '1/3 ounce toasted corn bits (such as CornNuts ®), crushed ADVERTISEMENT',
 'ADVERTISEMENT']

In [176]:
testlist = df['ingredients'][100000]
testlist

['1/2 small head green cabbage, cored and thinly sliced ADVERTISEMENT',
 '1/2 jicama, sliced into matchsticks ADVERTISEMENT',
 '1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT',
 '1/2 cup mayonnaise ADVERTISEMENT',
 '1/4 cup pineapple juice ADVERTISEMENT',
 '1 teaspoon white sugar ADVERTISEMENT',
 'hot sauce to taste ADVERTISEMENT',
 'salt and freshly ground black pepper to taste ADVERTISEMENT',
 '1/4 bunch chopped fresh cilantro ADVERTISEMENT',
 '1/3 ounce toasted corn bits (such as CornNuts ®), crushed ADVERTISEMENT',
 'ADVERTISEMENT']

In [177]:
# Scraping corpus of words related to measurements

page = requests.get('https://www.enchantedlearning.com/wordlist/measurement.shtml')
soup = BeautifulSoup(page.content, "html.parser")
measure_corpus = [tag.text for tag in soup.find_all('div',attrs={'class':'wordlist-item'})]
measure_corpus = measure_corpus + [text+'s' for text in measure_corpus] + ['taste','strip', 'strips', 'package' + 'packages' + 'satchet' + 'satchets' + 'sprigs','head','bunch','small','large','big','medium']

In [188]:
def clean_recipe(ingred_list):
    cleanedtext = []
    for ingred in ingred_list:
        # Ignore extra information
        matchtext = re.sub(r' \([^)]*\)', '', ingred)
        
        # Obtain all before first comma
        if re.compile('^(.+?)(?=,)').search(matchtext) is not None:
            matchtext = re.compile('^(.+?)(?=,)').search(matchtext).group(1)
            matchtext = matchtext.replace(' ADVERTISEMENT','')
            
        # Ignore advertisement tag
        elif (ingred == 'ADVERTISEMENT'):
            continue
        
        # If all text, make sure no advertisement tag
        else:
            matchtext = matchtext.replace(' ADVERTISEMENT','')
        
        # Tokenize ingredient list
        tokenized = word_tokenize(matchtext)
        
        # Remove words likely to be stop words or measurements
        removed_stop = [w for w in tokenized if not w in measure_corpus]
        removed_stop = [w for w in removed_stop if not w in stop_words]
        
        # Filter adjectives and nouns
        ingred_words = lambda pos: pos[:2] in ['JJ','NN','NNS']
        ingreds = [word.lower() for (word, pos) in pos_tag(removed_stop) if ingred_words(pos)]
        
        # Convert to singular
        ingreds = [p.singular_noun(word) if p.singular_noun(word) else word for word in ingreds]
        
        # Remove common ingredients 
        common = ['water','salt','pepper']
        cleanedtext.append(ingreds)
        cleanedtext = [[ing for ing in ingreds if not any(word in common for word in ingreds)] for ingreds in cleanedtext]
        
        # Remove additional descriptors for long ingredient names
        cleanedtext = [ingreds[-2:] if len(ingreds) > 2 else ingreds for ingreds in cleanedtext]
   
    return [(' ').join(item) for item in cleanedtext if len(item)>0]

In [705]:
clean_recipe(df['ingredients'][10])

['chicken stock',
 'winter vegetable',
 'roasting chicken',
 'yellow onion',
 'carrot',
 'stalk leaf',
 'parsnip',
 'fresh parsley',
 'fresh thyme',
 'fresh dill',
 'garlic',
 'black peppercorn',
 'carrot',
 'parsnip',
 'sweet potato',
 'butternut squash',
 'olive oil',
 'flat-leaf parsley']

In [180]:
#regex test
# re.compile('(?<=\d )(?!.*\d)(.+?)(?=[^a-zA-Z ])').search('1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT').group(1)

In [236]:
df['ingredients'][2]

['1 1/2 cups dried black beans, picked over and rinsed',
 '8 cups water, plus 1/4 cup',
 '2 cloves garlic',
 '3 tablespoons vegetable oil',
 '1 large green bell pepper, finely chopped, reserving about 1 teaspoon for garnish',
 '1 small red onion, finely chopped, reserving about 1 teaspoon for garnish',
 '2 teaspoons ground cumin',
 '2 tablespoons cider vinegar',
 'Salt and pepper',
 '1 plum tomato, finely chopped',
 '1 small red chile, finely chopped',
 'Fresh cilantro leaves, for garnish',
 'Dollop sour cream, for garnish',
 'Serving Suggestion: Tortilla chips']

In [195]:
# Clean recipes across dataframe

df['ingreds_list'] = df['ingredients'].apply(clean_recipe)

In [198]:
df['ingreds_list'].head(20)

0     [celery, green onion, parsley, crabmeat, crack...
1     [skirt steak, applewood bacon, red onion, jala...
2     [black bean, clove garlic, vegetable oil, red ...
3     [ground chuck, tomato sauce, egg noodle, sour ...
4     [rice, 1/2-cup quinoa, white tuna, seaweed kel...
5     [olive oil, italian eggplant, hummu hummu, gra...
6     [saffron thread, sugar, orange zest, carrot, u...
7     [hot-dog bun, unsalted butter, lobster lobster...
8     [olive oil, onion, pine nut, white rice, tomat...
9     [cauliflower, clove garlic, unsalted butter, b...
10    [chicken stock, winter vegetable, roasting chi...
11    [chili sauce, rice vinegar, fish sauce, fresh ...
12                        [cheese product, green chily]
13    [london broil, olive oil, worcestershire sauce...
14    [duck, garlic clove, cinnamon stick, fat lard,...
15    [purpose flour, cornmeal, baking powder, sugar...
16    [boneles, egg, bread crumb, deli ham, wine sau...
17    [tortilla chip, neely bbq, pork, bbq sauce

In [202]:
list_ingreds = df['ingreds_list']

In [220]:
combinations = []

for lists in tqdm(list_ingreds):
    combinations = combinations + list(itertools.combinations(lists,2))
    
combinations_df = pd.DataFrame(combinations)

100%|██████████| 122971/122971 [11:20:42<00:00,  3.01it/s]     


In [678]:
# List of top 200 ingredients on both sides of pairings

combinations_df.columns = ['ingred1','ingred2']
top_ingred1 = combinations_df['ingred1'].value_counts()[:200]
top_ingred2 = combinations_df['ingred2'].value_counts()[:200]

In [679]:
# Create new list of ranked ingredients by combining both sides of pairing list

top_ingreds = pd.DataFrame(top_ingred1).join(pd.DataFrame(top_ingred2),how='outer').fillna(0)
top_ingreds['total'] = top_ingreds['ingred1'] + top_ingreds['ingred2']
top_ingreds = top_ingreds.sort_values(by="total", ascending=False)['total']
top_ingreds = top_ingreds.reset_index() 
top_ingreds.columns = ['ingredient','total']

In [722]:
len(combinations_df)

5839145

In [680]:
top_ingreds_list = list(top_ingreds['ingredient'])

In [681]:
top_ingreds_list

['olive oil',
 'sugar',
 'egg',
 'all-purpose flour',
 'clove garlic',
 'butter',
 'unsalted butter',
 'onion',
 'vanilla extract',
 'vegetable oil',
 'brown sugar',
 'lemon juice',
 'milk',
 'white sugar',
 'garlic clove',
 'baking powder',
 'ground cinnamon',
 'carrot',
 'heavy cream',
 'garlic',
 'red onion',
 'baking soda',
 'bay leaf',
 'shallot',
 'wine vinegar',
 'honey',
 'chicken broth',
 'soy sauce',
 'tomato',
 'canola oil',
 'sour cream',
 'white wine',
 'ground cumin',
 'egg yolk',
 'lemon',
 'lime juice',
 'chicken stock',
 'dijon mustard',
 'confectioner sugar',
 'green onion',
 'scallion',
 'fresh parsley',
 'mayonnaise',
 'fresh thyme',
 'red bell',
 'paprika',
 'fresh cilantro',
 'cornstarch',
 'worcestershire sauce',
 'whole milk',
 'garlic powder',
 'egg white',
 'chili powder',
 'cream cheese',
 'parmesan cheese',
 'cilantro leaf',
 'dried oregano',
 'parsley leaf',
 'fresh ginger',
 'basil leaf',
 'flour',
 'parmesan',
 'orange juice',
 'ground nutmeg',
 'stick bu

In [682]:
# Filter for combinations with top ingredients

filtered_combinations = []

for x in tqdm(combinations_df.itertuples()):
    if x[1] in top_ingreds_list and x[2] in top_ingreds_list:
        filtered_combinations.append(x[1:3])

filtered_combinations_df = pd.DataFrame(filtered_combinations, columns=['ingred1','ingred2'])


0it [00:00, ?it/s]
2955it [00:00, 7950.52it/s]
8243it [00:00, 9707.03it/s]
24762it [00:00, 13526.53it/s]
45703it [00:00, 18803.08it/s]
68706it [00:00, 25952.34it/s]
88241it [00:01, 35077.58it/s]
112407it [00:01, 47176.06it/s]
137563it [00:01, 62380.45it/s]
161651it [00:01, 80212.00it/s]
183081it [00:01, 97418.09it/s]
203934it [00:01, 114707.75it/s]
228734it [00:01, 136758.15it/s]
254225it [00:01, 158845.54it/s]
279778it [00:01, 179183.42it/s]
304101it [00:01, 194551.68it/s]
329266it [00:02, 208761.26it/s]
353530it [00:02, 216313.99it/s]
377602it [00:02, 203106.00it/s]
399786it [00:02, 203052.23it/s]
423438it [00:02, 212053.48it/s]
447799it [00:02, 220626.08it/s]
470667it [00:02, 217245.33it/s]
492967it [00:02, 204535.92it/s]
513976it [00:02, 205300.59it/s]
536128it [00:03, 209910.77it/s]
557424it [00:03, 210814.60it/s]
579287it [00:03, 213098.64it/s]
600750it [00:03, 212408.91it/s]
627399it [00:03, 226177.86it/s]
652859it [00:03, 234013.17it/s]
676554it [00:03, 227084.59it/s]
701426it

In [683]:
filtered_combinations_df.head()

,ingred1,ingred2
0,celery,green onion
1,celery,parsley
2,celery,dry mustard
3,celery,hot sauce
4,celery,heavy cream


In [684]:
# Get count of recipes each top ingredient appears in
# Later determines radius/color of network

top_ingreds_recipes = dict.fromkeys(top_ingreds_list, 0)

for ingred in tqdm(top_ingreds_list):
    for lst in df['ingreds_list']:
        if ingred in lst:
            top_ingreds_recipes[ingred] += 1

top_ingreds_recipes


100%|██████████| 229/229 [00:10<00:00, 21.72it/s]

{'all-purpose flour': 19562,
 'almond': 2182,
 'almond extract': 1064,
 'apple': 1182,
 'avocado': 1371,
 'bacon': 1152,
 'baking powder': 7949,
 'baking soda': 6250,
 'balsamic vinegar': 2800,
 'banana': 1231,
 'basil leaf': 2945,
 'bay leaf': 4656,
 'beef broth': 980,
 'bittersweet chocolate': 1192,
 'black bean': 1280,
 'black olive': 1121,
 'black peppercorn': 1633,
 'blue cheese': 770,
 'boneles': 904,
 'bread crumb': 2988,
 'brown sugar': 11073,
 'butter': 19296,
 'buttermilk': 2460,
 'butternut squash': 699,
 'cake flour': 1000,
 'cake mix': 825,
 'canola oil': 3871,
 'caper': 1440,
 'carrot': 6529,
 'cayenne': 828,
 'celery': 1277,
 'celery stalk': 885,
 'cheddar cheese': 3096,
 'cherry tomato': 1423,
 'chicken': 809,
 'chicken breast': 1212,
 'chicken broth': 5640,
 'chicken stock': 3494,
 'chile powder': 672,
 'chili powder': 2840,
 'chive': 1079,
 'chocolate chip': 2696,
 'chopped onion': 2469,
 'cider vinegar': 2731,
 'cilantro': 1410,
 'cilantro leaf': 2367,
 'cinnamon': 1

In [685]:
top_ingreds_recipes_df = pd.DataFrame(list(zip(top_ingreds_recipes.keys(),top_ingreds_recipes.values())), columns = ['ingredient','recipes'])

top_ingreds = pd.merge(top_ingreds, top_ingreds_recipes_df)

In [686]:
top_ingreds['total'] = top_ingreds['total'].astype('int')

In [687]:
top_ingreds

,ingredient,total,recipes
0,olive oil,377461,32429
1,sugar,288310,21456
2,egg,257308,24435
3,all-purpose flour,218468,19562
4,clove garlic,216540,17952
5,butter,205882,19296
6,unsalted butter,184737,14839
7,onion,169772,15351
8,vanilla extract,142113,11957
9,vegetable oil,126008,11680


### Categories to rename or combine

lime + lime wedge
lemon + lemon wedge
vegetable oil + oil
potato + russet potato
lemon zest + lemon peel
lemon juice + juice lemon
skinles + boneles + chicken breast
sugar + white sugar
clove garlic + garlic clove + garlic
butter + unsalted butter + stick butter
chicken broth + chicken stock
parmesan cheese + parmesan
onion + yellow onion + chopped onion + diced onion
stalk celery + celery + celery stalk + rib celery
all-purpose flour + purpose flour
fresh ginger + ginger
slice bacon + bacon
cilantro leaf + fresh cilantro + cilantro
parsley leaf + flat-leaf parsley + fresh parsley
basil leaf + fresh basil
mint leaf + fresh mint
thyme leaf + fresh thyme

In [688]:
# Function to combine rows with similar names into single row with aggregated column

def combine_row(df, column1, aggcolumn1, aggcolumn2, commonvals, newname):
    tot1 = 0
    tot2 = 0
    df = df.append(pd.DataFrame([[newname,0,0]], columns=df.columns),ignore_index=True)
    for val in commonvals:
        tot1 = tot1 + df[aggcolumn1][df[column1] == val].values
        tot2 = tot2 + df[aggcolumn2][df[column1] == val].values
        df = df[df[column1] != val]
    df.loc[df[column1] == newname, aggcolumn1] = tot1
    df.loc[df[column1] == newname, aggcolumn2] = tot2
    return df

In [689]:
# Combine similar ingredients and remove unecesssary text

top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['lime','lime wedge'], 'lime_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['lemon','lemon wedge'], 'lemon_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['vegetable oil','oil'], 'vegetable oil_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['potato','russet potato'], 'potato_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['lemon zest','lemon peel'], 'lemon zest_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['lemon juice','juice lemon'], 'lemon juice_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['skinles','boneles','chicken breast'], 'chicken breast_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['slice bacon','bacon'], 'bacon_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['stalk celery', 'celery', 'celery stalk', 'rib celery'], 'celery_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['clove garlic','garlic clove','garlic'], 'garlic_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['sugar', 'white sugar'], 'sugar_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['butter','unsalted butter','stick butter'], 'butter_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['chicken broth', 'chicken stock'], 'chicken stock_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['onion', 'yellow onion', 'chopped onion', 'diced onion'], 'onion_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['parmesan cheese', 'parmesan'], 'parmesan_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['all-purpose flour','purpose flour'], 'all-purpose flour_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['fresh ginger','ginger'], 'ginger_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['cilantro leaf','fresh cilantro','cilantro'], 'cilantro_')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['basil leaf','fresh basil'], 'basil')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['thyme leaf','fresh thyme'], 'thyme')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['parsley leaf', 'flat-leaf parsley', 'fresh parsley'], 'parsley')
top_ingreds = combine_row(top_ingreds, 'ingredient', 'total', 'recipes', ['mint leaf','fresh mint'], 'mint_')

top_ingreds['ingredient'] = top_ingreds['ingredient'].str.replace('_', '')

top_ingreds = top_ingreds.sort_values(by="total", ascending=False).reset_index().drop('index',axis=1)
top_ingreds

,ingredient,total,recipes
0,butter,421929,36975
1,garlic,384999,32539
2,sugar,383695,31492
3,olive oil,377461,32429
4,egg,257308,24435
5,all-purpose flour,239413,21384
6,onion,229482,20845
7,vanilla extract,142113,11957
8,vegetable oil,138582,12645
9,lemon juice,127903,12743


In [690]:
top_ingreds = top_ingreds.drop_duplicates()
top_ingreds

,ingredient,total,recipes
0,butter,421929,36975
1,garlic,384999,32539
2,sugar,383695,31492
3,olive oil,377461,32429
4,egg,257308,24435
5,all-purpose flour,239413,21384
6,onion,229482,20845
7,vanilla extract,142113,11957
8,vegetable oil,138582,12645
9,lemon juice,127903,12743


In [691]:
combinations_df.to_csv('all_combinations.csv')
top_ingreds.to_csv('cleaned_top_ingreds.csv')
filtered_combinations_df.to_csv('filtered_combinations.csv')

In [692]:
# Count number of overlapping edges
filtered_combinations_df['ingred1'] = filtered_combinations_df['ingred1'].str.replace('clove garlic', 'garlic').replace('garlic clove','garlic').replace('white sugar','sugar').replace('unsalted butter','butter').replace('stick butter','butter').replace('chicken broth','chicken stock').replace('parmesan cheese','parmesan').replace('yellow onion','onion').replace('chopped onion','onion').replace('stalk celery','celery').replace('purpose flour','all-purpose flour').replace('fresh ginger','ginger').replace('slice bacon','bacon').replace('cilantro leaf','cilantro').replace('fresh cilantro','cilantro').replace('parsley leaf','parsley').replace('flat-leaf parsley','parsley').replace('fresh parsley','parsley').replace('basil leaf','basil').replace('fresh basil','basil').replace('thyme leaf','thyme').replace('fresh thyme','thyme').replace('boneles','chicken breast').replace('skinles','chicken breast').replace('celery stalk','celery').replace('diced onion','onion').replace('fresh mint','mint').replace('juice lemon','lemon juice').replace('lemon peel','lemon zest').replace('lemon wedge','lemon').replace('lime wedge','lime').replace('russet potato','potato').replace('mint leaf','mint').replace('oil','vegetable oil').replace('rib celery','celery')
filtered_combinations_df['ingred2'] = filtered_combinations_df['ingred2'].str.replace('clove garlic', 'garlic').replace('garlic clove','garlic').replace('white sugar','sugar').replace('unsalted butter','butter').replace('stick butter','butter').replace('chicken broth','chicken stock').replace('parmesan cheese','parmesan').replace('yellow onion','onion').replace('chopped onion','onion').replace('stalk celery','celery').replace('purpose flour','all-purpose flour').replace('fresh ginger','ginger').replace('slice bacon','bacon').replace('cilantro leaf','cilantro').replace('fresh cilantro','cilantro').replace('parsley leaf','parsley').replace('flat-leaf parsley','parsley').replace('fresh parsley','parsley').replace('basil leaf','basil').replace('fresh basil','basil').replace('thyme leaf','thyme').replace('fresh thyme','thyme').replace('boneles','chicken breast').replace('skinles','chicken breast').replace('celery stalk','celery').replace('diced onion','onion').replace('fresh mint','mint').replace('juice lemon','lemon juice').replace('lemon peel','lemon zest').replace('lemon wedge','lemon').replace('lime wedge','lime').replace('russet potato','potato').replace('mint leaf','mint').replace('oil','vegetable oil').replace('rib celery','celery')
filtered_combinations_df.head()

,ingred1,ingred2
0,celery,green onion
1,celery,parsley
2,celery,dry mustard
3,celery,hot sauce
4,celery,heavy cream


In [721]:
# Check if list of unique ingredients agree

set(top_ingreds['ingredient']) - set(filtered_combinations_df['ingred1'])

196

In [559]:
def swap_ingred(df, tup):
    index = tup[0]
    temp = df.loc[index,'ingred1']
    df.at[index, 'ingred1'] = df.loc[index, 'ingred2']
    df.at[index, 'ingred2'] = temp
    return df

In [570]:
# check 6 and 71

# for tup1 in tqdm(filtered_combinations_df.itertuples()):
#     for tup2 in filtered_combinations_df.loc[filtered_combinations_df.index > tup1[0]].itertuples():
#         if (tup1[1] == tup2[2]) & (tup1[2] == tup2[1]):
#             filtered_combinations_df = swap_ingred(filtered_combinations_df, tup2)

35it [00:54,  1.55s/it]

KeyboardInterrupt: 

In [556]:
test_tups = filtered_combinations_df[0:20]
test_tups = test_tups.append(pd.DataFrame([['butter','parsley']], columns = ['ingred1','ingred2']))
test_tups = test_tups.reset_index().drop('index', axis=1)
test_tups

,ingred1,ingred2
0,green onion,parsley
1,green onion,hot sauce
2,green onion,heavy cream
3,green onion,butter
4,parsley,hot sauce
5,parsley,heavy cream
6,parsley,butter
7,hot sauce,heavy cream
8,hot sauce,butter
9,heavy cream,butter


In [567]:
for tup1 in test_tups.itertuples():
    for tup2 in test_tups.loc[test_tups.index > tup1[0]].itertuples():
        if (tup1[1] == tup2[2]) & (tup1[2] == tup2[1]):
            test_tups = swap_ingred(test_tups, tup2)

In [568]:
test_tups

,ingred1,ingred2
0,green onion,parsley
1,green onion,hot sauce
2,green onion,heavy cream
3,green onion,butter
4,parsley,hot sauce
5,parsley,heavy cream
6,parsley,butter
7,hot sauce,heavy cream
8,hot sauce,butter
9,heavy cream,butter


In [550]:
test_tups.loc[6,'ingred1'] == test_tups.loc[20,'ingred2']

True

In [551]:
test_tups.loc[6,'ingred2'] == test_tups.loc[20,'ingred1']

True

In [696]:
filtered_combinations_df['pairs'] = list(zip(filtered_combinations_df['ingred1'], filtered_combinations_df['ingred2']))
filtered_combinations_df.head()

,ingred1,ingred2,pairs
0,celery,green onion,"(celery, green onion)"
1,celery,parsley,"(celery, parsley)"
2,celery,dry mustard,"(celery, dry mustard)"
3,celery,hot sauce,"(celery, hot sauce)"
4,celery,heavy cream,"(celery, heavy cream)"


In [697]:
# Group by pairing and get counts each pair occurs

filtered_pairs_df = filtered_combinations_df.groupby('pairs').agg({'ingred1':['count','first'],
                                                                  'ingred2':'first'}).reset_index()
filtered_pairs_df.columns = filtered_pairs_df.columns.droplevel(1)
filtered_pairs_df.columns = ['pairs','counts','ingred1','ingred2']
filtered_pairs_df.head()

,pairs,counts,ingred1,ingred2
0,"(all-purpose flour, all-purpose flour)",1471,all-purpose flour,all-purpose flour
1,"(all-purpose flour, almond)",480,all-purpose flour,almond
2,"(all-purpose flour, almond extract)",405,all-purpose flour,almond extract
3,"(all-purpose flour, apple)",203,all-purpose flour,apple
4,"(all-purpose flour, avocado)",41,all-purpose flour,avocado


In [698]:
# Drop same ingredients

for tup in tqdm(filtered_pairs_df.itertuples()):
    if tup[1][0] == tup[1][1]:
        filtered_pairs_df = filtered_pairs_df.drop(tup[0])


0it [00:00, ?it/s]
1710it [00:00, 15787.01it/s]
3034it [00:00, 14797.95it/s]
4237it [00:00, 13802.86it/s]
6044it [00:00, 14715.47it/s]
7441it [00:00, 14289.24it/s]
8751it [00:00, 13823.73it/s]
10625it [00:00, 14946.62it/s]
12717it [00:00, 16310.22it/s]
15053it [00:00, 17627.57it/s]
16855it [00:01, 17696.96it/s]
18886it [00:01, 18164.54it/s]
21030it [00:01, 18892.07it/s]
22934it [00:01, 18432.28it/s]
24790it [00:01, 16872.57it/s]
26512it [00:01, 14942.54it/s]
28071it [00:01, 14406.02it/s]
30003it [00:01, 15442.99it/s]
32061it [00:01, 16564.48it/s]
33929it [00:02, 16619.90it/s]

In [699]:
filtered_pairs_df.shape

(33735, 4)

In [700]:
# Clean up same pairs but different ordered

for tup1 in tqdm(filtered_pairs_df.itertuples()):
    for tup2 in filtered_pairs_df.loc[filtered_pairs_df.index > tup1[0]].itertuples():
        if (tup1[3] == tup2[4]) and (tup1[4] == tup2[3]):
            filtered_pairs_df.loc[tup1[0], 'counts'] += filtered_pairs_df.loc[tup2[0], 'counts']
            filtered_pairs_df = filtered_pairs_df.drop(tup2[0])


0it [00:00, ?it/s]
2it [00:00, 12.56it/s]
4it [00:00, 13.56it/s]
6it [00:00, 14.79it/s]
9it [00:00, 16.44it/s]
12it [00:00, 17.55it/s]
15it [00:00, 18.55it/s]
17it [00:00, 18.42it/s]
19it [00:01, 18.57it/s]
21it [00:01, 18.82it/s]
23it [00:01, 18.78it/s]
25it [00:01, 18.42it/s]
27it [00:01, 18.62it/s]
29it [00:01, 18.44it/s]
31it [00:01, 18.32it/s]
34it [00:01, 19.21it/s]
36it [00:01, 19.23it/s]
38it [00:02, 19.36it/s]
40it [00:02, 18.54it/s]
42it [00:02, 17.54it/s]
45it [00:02, 18.63it/s]
47it [00:02, 18.97it/s]
49it [00:02, 19.11it/s]
51it [00:02, 19.16it/s]
53it [00:02, 19.25it/s]
55it [00:02, 18.76it/s]
57it [00:03, 18.91it/s]
59it [00:03, 18.30it/s]
61it [00:03, 18.10it/s]
63it [00:03, 16.67it/s]
65it [00:03, 14.41it/s]
67it [00:03, 15.26it/s]
69it [00:03, 16.02it/s]
71it [00:03, 16.65it/s]
73it [00:04, 17.33it/s]
75it [00:04, 17.98it/s]
77it [00:04, 17.84it/s]
79it [00:04, 18.04it/s]
81it [00:04, 18.46it/s]
83it [00:04, 18.67it/s]
85it [00:04, 18.48it/s]
87it [00:04, 16.94it/s]


794it [00:38, 17.45it/s]
796it [00:38, 17.48it/s]
798it [00:39, 18.02it/s]
800it [00:39, 18.18it/s]
802it [00:39, 17.86it/s]
804it [00:39, 17.96it/s]
806it [00:39, 17.02it/s]
809it [00:39, 18.37it/s]
812it [00:39, 19.33it/s]
814it [00:39, 17.52it/s]
816it [00:40, 17.54it/s]
818it [00:40, 16.91it/s]
820it [00:40, 16.69it/s]
822it [00:40, 16.17it/s]
824it [00:40, 15.70it/s]
826it [00:40, 16.45it/s]
829it [00:40, 17.97it/s]
832it [00:40, 19.10it/s]
835it [00:41, 20.16it/s]
838it [00:41, 19.41it/s]
840it [00:41, 18.35it/s]
843it [00:41, 19.54it/s]
846it [00:41, 20.27it/s]
849it [00:41, 20.26it/s]
852it [00:41, 21.22it/s]
855it [00:42, 18.94it/s]
857it [00:42, 16.15it/s]
859it [00:42, 16.61it/s]
862it [00:42, 17.86it/s]
865it [00:42, 19.02it/s]
868it [00:42, 20.18it/s]
871it [00:42, 18.88it/s]
874it [00:43, 19.85it/s]
877it [00:43, 19.63it/s]
880it [00:43, 19.94it/s]
883it [00:43, 20.76it/s]
886it [00:43, 21.39it/s]
889it [00:43, 19.37it/s]
891it [00:43, 17.95it/s]
893it [00:44, 17.05it/s]


1629it [01:15, 23.08it/s]
1632it [01:16, 23.10it/s]
1635it [01:16, 23.16it/s]
1638it [01:16, 23.14it/s]
1641it [01:16, 23.03it/s]
1644it [01:16, 23.29it/s]
1647it [01:16, 23.40it/s]
1650it [01:16, 23.29it/s]
1653it [01:17, 23.41it/s]
1656it [01:17, 23.59it/s]
1659it [01:17, 23.87it/s]
1662it [01:17, 23.52it/s]
1665it [01:17, 23.38it/s]
1668it [01:17, 23.35it/s]
1671it [01:17, 23.46it/s]
1674it [01:17, 22.37it/s]
1677it [01:18, 22.37it/s]
1680it [01:18, 23.31it/s]
1683it [01:18, 23.20it/s]
1686it [01:18, 22.97it/s]
1689it [01:18, 23.47it/s]
1692it [01:18, 23.66it/s]
1696it [01:18, 25.48it/s]
1699it [01:18, 26.26it/s]
1702it [01:19, 26.00it/s]
1705it [01:19, 25.23it/s]
1708it [01:19, 24.38it/s]
1711it [01:19, 24.04it/s]
1714it [01:19, 24.08it/s]
1717it [01:19, 23.89it/s]
1720it [01:19, 24.05it/s]
1723it [01:19, 23.83it/s]
1726it [01:20, 23.66it/s]
1729it [01:20, 23.47it/s]
1732it [01:20, 23.30it/s]
1735it [01:20, 23.10it/s]
1738it [01:20, 23.32it/s]
1741it [01:20, 23.47it/s]
1744it [01:2

2476it [01:51, 24.58it/s]
2479it [01:51, 24.37it/s]
2482it [01:51, 24.31it/s]
2485it [01:51, 24.31it/s]
2488it [01:51, 24.64it/s]
2491it [01:51, 24.47it/s]
2494it [01:52, 24.11it/s]
2497it [01:52, 24.23it/s]
2500it [01:52, 24.17it/s]
2503it [01:52, 24.13it/s]
2506it [01:52, 24.14it/s]
2509it [01:52, 24.11it/s]
2512it [01:52, 24.15it/s]
2515it [01:52, 23.73it/s]
2518it [01:53, 23.88it/s]
2521it [01:53, 24.02it/s]
2524it [01:53, 25.04it/s]
2527it [01:53, 25.99it/s]
2530it [01:53, 26.71it/s]
2533it [01:53, 26.76it/s]
2536it [01:53, 25.48it/s]
2539it [01:53, 25.25it/s]
2542it [01:53, 24.87it/s]
2545it [01:54, 24.65it/s]
2548it [01:54, 24.14it/s]
2551it [01:54, 23.88it/s]
2554it [01:54, 24.14it/s]
2557it [01:54, 23.56it/s]
2560it [01:54, 23.88it/s]
2563it [01:54, 23.90it/s]
2566it [01:54, 24.50it/s]
2569it [01:55, 24.56it/s]
2572it [01:55, 24.33it/s]
2575it [01:55, 24.24it/s]
2578it [01:55, 24.33it/s]
2581it [01:55, 24.40it/s]
2584it [01:55, 24.36it/s]
2587it [01:55, 24.10it/s]
2590it [01:5

3335it [02:25, 24.58it/s]
3338it [02:25, 24.57it/s]
3341it [02:25, 24.69it/s]
3344it [02:25, 24.78it/s]
3347it [02:26, 24.95it/s]
3350it [02:26, 24.70it/s]
3353it [02:26, 24.79it/s]
3356it [02:26, 24.50it/s]
3359it [02:26, 24.77it/s]
3362it [02:26, 24.68it/s]
3365it [02:26, 24.90it/s]
3368it [02:26, 25.03it/s]
3371it [02:27, 25.11it/s]
3374it [02:27, 24.96it/s]
3377it [02:27, 24.85it/s]
3380it [02:27, 24.97it/s]
3383it [02:27, 25.36it/s]
3387it [02:27, 27.23it/s]
3391it [02:27, 28.65it/s]
3395it [02:27, 29.77it/s]
3399it [02:28, 30.86it/s]
3403it [02:28, 31.22it/s]
3407it [02:28, 29.35it/s]
3410it [02:28, 28.07it/s]
3413it [02:28, 27.22it/s]
3416it [02:28, 26.32it/s]
3419it [02:28, 25.85it/s]
3422it [02:28, 25.35it/s]
3425it [02:29, 25.03it/s]
3428it [02:29, 24.84it/s]
3431it [02:29, 24.76it/s]
3434it [02:29, 24.67it/s]
3437it [02:29, 24.74it/s]
3440it [02:29, 24.39it/s]
3443it [02:29, 24.39it/s]
3446it [02:29, 24.59it/s]
3449it [02:29, 25.12it/s]
3452it [02:30, 25.03it/s]
3455it [02:3

...

13655it [07:21, 50.82it/s]
13661it [07:21, 49.67it/s]
13667it [07:21, 50.07it/s]
13673it [07:21, 49.29it/s]
13679it [07:22, 51.76it/s]
13686it [07:22, 55.84it/s]
13693it [07:22, 58.32it/s]
13700it [07:22, 60.23it/s]
13707it [07:22, 62.20it/s]
13714it [07:22, 63.87it/s]
13722it [07:22, 66.05it/s]
13729it [07:22, 67.12it/s]
13736it [07:22, 67.38it/s]
13743it [07:22, 64.56it/s]
13750it [07:23, 59.22it/s]
13757it [07:23, 56.08it/s]
13763it [07:23, 53.38it/s]
13769it [07:23, 52.51it/s]
13775it [07:23, 51.64it/s]
13781it [07:23, 50.51it/s]
13787it [07:23, 49.63it/s]
13792it [07:23, 48.92it/s]
13797it [07:24, 48.94it/s]
13802it [07:24, 48.26it/s]
13807it [07:24, 47.43it/s]
13812it [07:24, 47.61it/s]
13817it [07:24, 46.96it/s]
13822it [07:24, 47.17it/s]
13827it [07:24, 46.98it/s]
13832it [07:24, 47.62it/s]
13837it [07:24, 47.57it/s]
13844it [07:25, 52.46it/s]
13851it [07:25, 56.22it/s]
13858it [07:25, 59.57it/s]
13866it [07:25, 62.68it/s]
13874it [07:25, 65.23it/s]
13882it [07:25, 67.12it/s]
1

15562it [07:52, 56.93it/s]
15569it [07:53, 60.12it/s]
15578it [07:53, 65.78it/s]
15585it [07:53, 66.82it/s]
15592it [07:53, 64.57it/s]
15599it [07:53, 66.09it/s]
15607it [07:53, 68.58it/s]
15615it [07:53, 70.57it/s]
15623it [07:53, 72.32it/s]
15631it [07:53, 73.30it/s]
15639it [07:53, 74.73it/s]
15647it [07:54, 70.54it/s]
15655it [07:54, 64.78it/s]
15662it [07:54, 60.93it/s]
15669it [07:54, 58.12it/s]
15675it [07:54, 55.83it/s]
15681it [07:54, 55.62it/s]
15688it [07:54, 57.14it/s]
15694it [07:54, 56.44it/s]
15700it [07:55, 56.86it/s]
15706it [07:55, 56.48it/s]
15712it [07:55, 56.90it/s]
15718it [07:55, 56.16it/s]
15724it [07:55, 55.58it/s]
15730it [07:55, 54.85it/s]
15737it [07:55, 58.13it/s]
15746it [07:55, 63.93it/s]
15754it [07:55, 67.50it/s]
15763it [07:56, 72.69it/s]
15773it [07:56, 78.97it/s]
15783it [07:56, 83.64it/s]
15793it [07:56, 87.18it/s]
15803it [07:56, 90.35it/s]
15813it [07:56, 91.68it/s]
15823it [07:56, 89.19it/s]
15833it [07:56, 81.69it/s]
15842it [07:56, 77.37it/s]
1

17909it [08:24, 66.80it/s]
17916it [08:24, 64.12it/s]
17923it [08:24, 63.46it/s]
17930it [08:24, 58.95it/s]
17937it [08:24, 60.41it/s]
17944it [08:24, 61.35it/s]
17954it [08:24, 69.35it/s]
17965it [08:24, 77.52it/s]
17977it [08:24, 86.02it/s]
17989it [08:25, 93.01it/s]
18000it [08:25, 96.94it/s]
18011it [08:25, 98.65it/s]
18023it [08:25, 102.58it/s]
18034it [08:25, 104.13it/s]
18045it [08:25, 105.40it/s]
18056it [08:25, 93.79it/s] 
18066it [08:25, 83.31it/s]
18075it [08:26, 76.56it/s]
18084it [08:26, 73.80it/s]
18092it [08:26, 70.68it/s]
18100it [08:26, 69.38it/s]
18108it [08:26, 68.38it/s]
18115it [08:26, 66.83it/s]
18122it [08:26, 66.22it/s]
18129it [08:26, 65.76it/s]
18136it [08:26, 65.23it/s]
18146it [08:27, 72.47it/s]
18158it [08:27, 81.17it/s]
18169it [08:27, 87.06it/s]
18180it [08:27, 92.02it/s]
18191it [08:27, 95.99it/s]
18203it [08:27, 100.53it/s]
18214it [08:27, 102.97it/s]
18226it [08:27, 105.75it/s]
18237it [08:27, 104.73it/s]
18248it [08:28, 90.92it/s] 
18258it [08:28, 82.

20964it [08:55, 69.10it/s]
20972it [08:56, 63.42it/s]
20980it [08:56, 65.08it/s]
20989it [08:56, 70.67it/s]
20999it [08:56, 76.34it/s]
21008it [08:56, 78.07it/s]
21018it [08:56, 82.35it/s]
21028it [08:56, 85.26it/s]
21038it [08:56, 88.28it/s]
21048it [08:56, 88.05it/s]
21057it [08:57, 86.13it/s]
21066it [08:57, 85.94it/s]
21076it [08:57, 88.94it/s]
21085it [08:57, 85.29it/s]
21094it [08:57, 82.36it/s]
21103it [08:57, 69.06it/s]
21111it [08:57, 63.56it/s]
21118it [08:57, 59.72it/s]
21125it [08:58, 57.25it/s]
21131it [08:58, 56.07it/s]
21137it [08:58, 54.73it/s]
21143it [08:58, 52.58it/s]
21149it [08:58, 51.46it/s]
21155it [08:58, 49.88it/s]
21161it [08:58, 50.81it/s]
21170it [08:58, 57.41it/s]
21179it [08:59, 63.88it/s]
21188it [08:59, 69.44it/s]
21197it [08:59, 74.36it/s]
21206it [08:59, 77.10it/s]
21216it [08:59, 80.47it/s]
21226it [08:59, 83.92it/s]
21235it [08:59, 79.82it/s]
21245it [08:59, 84.13it/s]
21254it [08:59, 85.33it/s]
21263it [08:59, 83.66it/s]
21273it [09:00, 86.80it/s]
2

25301it [09:27, 224.30it/s]
25325it [09:28, 177.63it/s]
25346it [09:28, 156.05it/s]
25373it [09:28, 178.07it/s]
25400it [09:28, 197.21it/s]
25427it [09:28, 213.79it/s]
25456it [09:28, 230.83it/s]
25482it [09:28, 234.66it/s]
25507it [09:28, 182.59it/s]
25528it [09:29, 166.27it/s]
25556it [09:29, 188.73it/s]
25585it [09:29, 210.53it/s]
25615it [09:29, 231.16it/s]
25644it [09:29, 244.67it/s]
25671it [09:29, 242.55it/s]
25697it [09:29, 189.44it/s]
25719it [09:30, 185.24it/s]
25749it [09:30, 208.76it/s]
25779it [09:30, 228.84it/s]
25810it [09:30, 246.95it/s]
25839it [09:30, 257.93it/s]
25867it [09:30, 201.51it/s]
25891it [09:30, 186.62it/s]
25921it [09:30, 209.76it/s]
25953it [09:30, 233.79it/s]
25985it [09:31, 253.28it/s]
26016it [09:31, 266.79it/s]
26045it [09:31, 218.24it/s]
26070it [09:31, 181.61it/s]
26100it [09:31, 204.70it/s]
26130it [09:31, 225.68it/s]
26160it [09:31, 243.08it/s]
26191it [09:31, 259.03it/s]
26219it [09:32, 246.52it/s]
26246it [09:32, 189.25it/s]
26268it [09:32, 182.

In [701]:
filtered_pairs_df.shape

(17833, 4)

In [702]:
filtered_pairs_df.head(200)

,pairs,counts,ingred1,ingred2
1,"(all-purpose flour, almond)",657,all-purpose flour,almond
2,"(all-purpose flour, almond extract)",634,all-purpose flour,almond extract
3,"(all-purpose flour, apple)",356,all-purpose flour,apple
4,"(all-purpose flour, avocado)",53,all-purpose flour,avocado
5,"(all-purpose flour, bacon)",411,all-purpose flour,bacon
6,"(all-purpose flour, baking powder)",6721,all-purpose flour,baking powder
7,"(all-purpose flour, baking soda)",5199,all-purpose flour,baking soda
8,"(all-purpose flour, balsamic vinegar)",141,all-purpose flour,balsamic vinegar
9,"(all-purpose flour, banana)",247,all-purpose flour,banana
10,"(all-purpose flour, basil)",466,all-purpose flour,basil


In [703]:
filtered_pairs_df.to_csv('filtered_pairs.csv')

In [704]:
len(df['ingreds_list'])

122971